In [1]:
import pandas as pd

In [2]:
def fix_volume(string):
    if string[-1:]=='M':
        string = float(string.replace('M', '').replace(',', '.'))
        string = int(string * 1000000)
    elif string[-1:]=='K':
        string = float(string.replace('K', '').replace(',', '.'))
        string = int(string * 1000)
    return string

In [3]:
frame_petroleo = pd.read_csv('D:\\Backup Daniel\\projetos\\bolsa\\historicos\\Petróleo Brent Futuros Dados Históricos.csv',
                    sep=',', parse_dates=['Data'], dayfirst=True)

frame_petroleo.rename(columns={
    'Data': 'data', 
    'Último':'fechamento', 
    'Abertura':'abertura', 
    'Máxima':'maxima',
    'Mínima':'minima',
    'Vol.':'volume',
    'Var%':'variacao'
}, inplace=True)

frame_petroleo['volume'] = frame_petroleo['volume'].apply(fix_volume)

frame_petroleo.drop(frame_petroleo[frame_petroleo.volume=='-'].index, inplace=True)

frame_petroleo['fechamento'] = frame_petroleo['fechamento'].apply(lambda x: (float(x.replace('.', '').replace(',', '.'))))
frame_petroleo['abertura'] = frame_petroleo['abertura'].apply(lambda x: (float(x.replace('.', '').replace(',', '.'))))
frame_petroleo['maxima'] = frame_petroleo['maxima'].apply(lambda x: (float(x.replace('.', '').replace(',', '.'))))
frame_petroleo['minima'] = frame_petroleo['minima'].apply(lambda x: (float(x.replace('.', '').replace(',', '.'))))
frame_petroleo['volume'] = frame_petroleo['volume'].astype('int64')

frame_petroleo.head()

,data,fechamento,abertura,maxima,minima,volume,variacao
1,2020-03-16,30.05,35.82,35.84,29.45,330120,"-6,82%"
3,2020-03-13,33.85,32.85,36.15,32.50,360630,"1,90%"
4,2020-03-12,33.22,35.99,36.46,32.50,472740,"-7,18%"
5,2020-03-11,35.79,37.27,39.70,35.35,471770,"-3,84%"
6,2020-03-10,37.22,35.84,38.22,35.00,551290,"8,32%"


In [4]:
frame_petr = pd.read_csv('D:\\Backup Daniel\\projetos\\bolsa\\historicos\\PETR4 Dados Históricos.csv',
                    sep=',', parse_dates=['Data'], dayfirst=True)

frame_petr.rename(columns={
    'Data': 'data', 
    'Último':'fechamento', 
    'Abertura':'abertura', 
    'Máxima':'maxima',
    'Mínima':'minima',
    'Vol.':'volume',
    'Var%':'variacao'
}, inplace=True)

frame_petr['volume'] = frame_petr['volume'].apply(fix_volume)

frame_petr.drop(frame_petr[frame_petr.volume=='-'].index, inplace=True)

frame_petr['fechamento'] = frame_petr['fechamento'].apply(lambda x: (float(x.replace('.', '').replace(',', '.'))))
frame_petr['abertura'] = frame_petr['abertura'].apply(lambda x: (float(x.replace('.', '').replace(',', '.'))))
frame_petr['maxima'] = frame_petr['maxima'].apply(lambda x: (float(x.replace('.', '').replace(',', '.'))))
frame_petr['minima'] = frame_petr['minima'].apply(lambda x: (float(x.replace('.', '').replace(',', '.'))))
frame_petr['volume'] = frame_petr['volume'].astype('int64')

frame_petr.head()

,data,fechamento,abertura,maxima,minima,volume,variacao
1,2020-03-13,15.40,15.10,15.40,12.70,173050000,"22,22%"
2,2020-03-12,12.60,13.66,13.86,11.08,149990000,"-20,50%"
3,2020-03-11,15.85,16.70,17.13,15.00,129759999,"-9,74%"
4,2020-03-10,17.56,18.65,18.65,16.37,169490000,"9,41%"
5,2020-03-09,16.05,17.81,18.23,15.41,252820000,"-29,70%"


In [5]:
frame = pd.merge(
    frame_petr,
    frame_petroleo,
    how='inner',
    left_on='data',
    right_on='data'
)

frame.rename(columns={
    'fechamento_x':'fechamento_petr',
    'abertura_x': 'abertura_petr',
    'maxima_x': 'maxima_petr',
    'minima_x': 'minima_petr',
    'volume_x': 'volume_petr',
    'variacao_x': 'variacao_petr',
    'fechamento_y':'fechamento_petroleo',
    'abertura_y': 'abertura_petroleo',
    'maxima_y': 'maxima_petroleo',
    'minima_y': 'minima_petroleo',
    'volume_y': 'volume_petroleo',
    'variacao_y': 'variacao_petroleo'},
            inplace=True)
frame.head()

,data,fechamento_petr,abertura_petr,maxima_petr,minima_petr,volume_petr,variacao_petr,fechamento_petroleo,abertura_petroleo,maxima_petroleo,minima_petroleo,volume_petroleo,variacao_petroleo
0,2020-03-13,15.40,15.10,15.40,12.70,173050000,"22,22%",33.85,32.85,36.15,32.50,360630,"1,90%"
1,2020-03-12,12.60,13.66,13.86,11.08,149990000,"-20,50%",33.22,35.99,36.46,32.50,472740,"-7,18%"
2,2020-03-11,15.85,16.70,17.13,15.00,129759999,"-9,74%",35.79,37.27,39.70,35.35,471770,"-3,84%"
3,2020-03-10,17.56,18.65,18.65,16.37,169490000,"9,41%",37.22,35.84,38.22,35.00,551290,"8,32%"
4,2020-03-09,16.05,17.81,18.23,15.41,252820000,"-29,70%",34.36,38.28,38.34,31.02,779720,"-24,10%"


In [6]:
frame.to_parquet('D:\\Backup Daniel\\projetos\\bolsa\\historicos\\dados.parquet', compression='gzip', index=False)